In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('../ipsim'))
if module_path not in sys.path:
    sys.path.append(module_path)

from ipsim import *

import numpy as np
import random as rnd
import matplotlib.pyplot as plt

In [ ]:
class ECSRTTest:
    Ea  = 72750     # activation energy J/gmol
    R   = 8.314     # gas constant J/gmol/K
    k0  = 7.2e10    # Arrhenius rate constant 1/min
    dHr = -5.0e4    # Enthalpy of reaction [J/mol]
    rho = 1000.0    # Density [g/L]
    Cp  = 0.239     # Heat capacity [J/g/K]
    UA  = 5.0e4     # Heat transfer [J/min/K]
    
    V   = 100.0     # Volume [L]
    q   = 100.0     # Flowrate [L/min]
    cAi = 1.0       # Inlet feed concentration [mol/L]
    Ti  = 350.0     # Inlet feed temperature [K]
    Tc  = 300.0     # Coolant temperature [K]

    cA0 = 0.5;      # Initial concentration [mol/L]
    T0  = 350.0;    # Initial temperature [K]
    
    def k(T):
        return ECSRTTest.k0*np.exp(-ECSRTTest.Ea/ECSRTTest.R/T)

In [ ]:
class ExothermicContinuousStirredTankReactor0(ProcessNode):
    def __init__(self, name, *, cA0 = ECSRTTest.cA0, T0 = ECSRTTest.T0, V = ECSRTTest.V
                , dHr = ECSRTTest.dHr, rho = ECSRTTest.rho, Cp=ECSRTTest.Cp,UA = ECSRTTest.UA):
        super().__init__(name)
        self._cA = cA0
        self._T = T0
        self._V = V
        self._dHr = dHr
        self._rho=rho
        self._Cp = Cp
        self._UA = UA
        self.create_input("q")
        self.create_input("cAi")
        self.create_input("Ti")
        self.create_input("Tc")
        self.create_output("cA")
        self.create_output("T")
    
    def evaluate(self):
        i = self.inputs()
        q   = i["q"]()
        cAi = i["cAi"]()
        Ti  = i["Ti"]()
        Tc  = i["Tc"]()
        cA  = self._cA
        T   = self._T
        
        kcA = ECSRTTest.k(T)*cA
        dt = self._model.dt()

        qV    = q/self._V
        dHpC  = -self._dHr/(self._rho*self._Cp)
        UAVpC = self._UA/(self._V*self._rho*self._Cp)
     
        _dcA  = (qV*(cAi-cA) - kcA)*dt
        _dT  = (qV*(Ti-T)+dHpC*kcA+UAVpC*(Tc-T))*dt
        cA = cA + _dcA
        T  = T  + _dT
        
        self._cA = cA
        self._T = T

        self.set_result("cA",cA)
        self.set_result("T",T)

In [ ]:
processModel = ProcessModel("test",dt=0.005)
processModel.add_node(ProcessInputNode("InletFeed", {"q":100,"cAi":1,"Ti":350}))
processModel.add_node(ProcessInputNode("Coolant", {"Tc":300}))
processModel.add_node(ExothermicContinuousStirredTankReactor0("ECSTR"))
processModel.bond_nodes("ECSTR","q","InletFeed","q")
processModel.bond_nodes("ECSTR","cAi","InletFeed","cAi")
processModel.bond_nodes("ECSTR","Ti","InletFeed","Ti")
processModel.bond_nodes("ECSTR","Tc","Coolant","Tc")

In [ ]:
processModel.reset_time(0.005)

In [ ]:
cAs  = []
Ts   = []
for i in range(5000):
    state = processModel.next_state(("ECSTR",))
    cAs.append(state['ECSTR']['cA'])
    Ts.append(state['ECSTR']['T'])
        
plt.figure(figsize=(12,3))
plt.subplot(1, 2, 1)
plt.plot(cAs)
plt.title('Concentration [mol/L]')
plt.subplot(1, 2, 2)
plt.plot(Ts)
plt.title('Temperature [K]')
plt.show()